In [ ]:
# Adapted from Lecture 8 Code

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd

#Upload the fakeTestDataset.csv file to google collab
df = pd.read_csv('fakeTestDataset.csv')

#Using a dictionary made adapting from the iris set program easier
madeUpData = {
    #Uses the pandas dataframe to assign the first 3 columns as a nested list
    'data': df.iloc[:, :3].values.tolist(),
    #Uses the pandas dataframe to assign the output index column
    'target': df['Book Index'].values.tolist()
}

X, y = madeUpData['data'], madeUpData['target'] ## Dataset Size X 5 ## Dataset Size X 1

                                                     ## 20% of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train) # uses mean and standard deviation of the data to move the data
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.long)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.long) # Long used because other functions will be expecting it

class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

input_size = X_train.shape[1] # 3
hidden_size = 8 # just left this number for now
num_classes = 4 # 4 possible books as outputs
model = MLP(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()

#Adam optimizer with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr = 0.001)

batch_size = 8 # left alone for now
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

#30 epochs was randomly assigned for now
num_epochs = 30
for epoch in range(num_epochs):
  for batch_X, batch_y in train_loader: # Process each batch separately

    # Forward pass
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
with torch.no_grad():
  outputs = model(X_test)
  _, predicted = torch.max(outputs, 1)

accuracy = accuracy_score(y_test, predicted)
print(f'Accuracy: {accuracy * 100:.2f}%')



Epoch [10/30], Loss: 1.2876
Epoch [20/30], Loss: 1.1405
Epoch [30/30], Loss: 1.0959
Accuracy: 37.50%


In [ ]:
#Little test for pandas
import pandas as pd

df = pd.read_csv('fakeTestDataset.csv')

combined_row_data = df['Book Index'].values.tolist()
print(combined_row_data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
